In [13]:
import pandas as pd
from antropy import lziv_complexity
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np
import argparse
import mne.io
import random
import mne
import os
import multiprocessing as mp
import neurokit2 as nk
from scipy.fft import fft, ifft
from scipy.signal import hilbert
from scipy.fftpack import rfft, irfft

from joblib import Parallel, delayed
import glob


In [14]:
def get_pred(trial, epochs):
    Lyaps = []
    Dims = []
    Ent = []
    LZC = []
    KDF = []

    nr_channels =  epochs.shape[1]
    trial_data = epochs[trial]

    for ch in range(nr_channels):
        channel_data = trial_data[ch]
        lle, _ = nk.complexity_lyapunov(channel_data, method="rosenstein1993", show=False)
        Lyaps.append(lle)

        dims, _ = nk.complexity_dimension(channel_data)
        Dims.append(dims)

        ent, _ = nk.entropy_multiscale(channel_data, show=False, dimension=dims)
        Ent.append(ent)

        lzc, _ = nk.complexity_lempelziv(channel_data, show=False)
        LZC.append(lzc)

        kdf, _ = nk.fractal_katz(channel_data)
        KDF.append(kdf)

    print('Done Trial {}'.format(str(trial)))

    return Lyaps, Dims, Ent, LZC, KDF



In [ ]:
out_dir = 'data/output/Pred/'
in_dir = 'data/input/10s/'
lfreq = 0.1
hfreq = 40

# output
Lyaps_max = []
Lyaps_space = []
Dims_mean = []
Dims_space = []
Ent_mean = []
Ent_space = []
LZC_mean = []
LZC_space = []
KDF_mean = []
KDF_space = []

sub_all = []
day_all = []   
condition_all = []



# make output directory
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

paths = glob.glob(in_dir + '*.fif')
paths.sort()
paths = paths[:2]

#loop over all conditions and particiants
for i, path in enumerate(paths):
    sub = os.path.basename(path)[:4]
    day = os.path.basename(path)[5:9]
    condition = os.path.basename(path)[10:-8]

    print(f"Analyzing Lyapunov Exponenf of {path}");

    #################################
    #          LOAD  DATA          #
    #################################

    epochs_mne = mne.read_epochs(path, preload=True)
    epochs_mne.interpolate_bads(reset_bads=True)
    epochs_mne.pick_types(eeg=True, meg=False, stim=False, eog=False, ecg=False, emg=False, misc=False, exclude='bads')

    epochs = epochs_mne.get_data()
    fs = 256 #check this


    # prepare data
    #epochs_res = epochs_mne.resample(250)
    epochs_filt = epochs_mne.filter(lfreq, hfreq, verbose = False)

    # if data is too long only use the first 3 min of data
    nr_trials = min([len(epochs_filt),30])
    epochs = epochs_filt.get_data()
    nr_channels =  epochs.shape[1]

    #################################
    #    Calculate LZC             #
    #################################

    # Run parallel processing for all trials
    results = Parallel(n_jobs=-1)(
        delayed(get_pred)(trial, epochs) for trial in range(nr_trials)
    )

    # Convert results to NumPy array
    results = np.array(results)

    # Compute summary statistics
    Lyaps_max.append(np.mean(results[:, 0, :], axis=(0, 1)))
    Lyaps_space.append(np.mean(results[:, 0, :], axis=0))
    Dims_mean.append(np.mean(results[:, 1, :], axis=(0, 1)))
    Dims_space.append(np.mean(results[:, 1, :], axis=0))
    Ent_mean.append(np.mean(results[:, 2, :], axis=(0, 1)))
    Ent_space.append(np.mean(results[:, 2, :], axis=0))
    LZC_mean.append(np.mean(results[:, 3, :], axis=(0, 1)))
    LZC_space.append(np.mean(results[:, 3, :], axis=0))
    KDF_mean.append(np.mean(results[:, 4, :], axis=(0, 1)))
    KDF_space.append(np.mean(results[:, 4, :], axis=0))
    sub_all.append(sub)
    day_all.append(day)
    condition_all.append(condition)


    # save part
    output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all,
                'Lyaps_max':Lyaps_max, 'Dims_mean':Dims_mean,
                'Ent_mean':Ent_mean, 'LZC_mean':LZC_mean,'KDF_mean':KDF_mean }
    output_df = pd.DataFrame(output_df)
    output_df.to_csv(f'{out_dir}/Pred_{lfreq}_{hfreq}.csv', index=False, sep=',')

    # save space
    output_df_space = {'sub':sub_all, 'day': day_all,'condition':condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(Lyaps_space).reset_index(drop=True)), axis = 1)
    output_df.to_csv(f'{out_dir}/Lyaps_space_{lfreq}_{hfreq}.csv', index=False, sep=',')

    # save space
    output_df_space = {'sub':sub_all, 'day': day_all,'condition':condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(Dims_space).reset_index(drop=True)), axis = 1)
    output_df.to_csv(f'{out_dir}/Dims_space_{lfreq}_{hfreq}.csv', index=False, sep=',')

    # save space
    output_df_space = {'sub':sub_all, 'day': day_all,'condition':condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(Ent_space).reset_index(drop=True)), axis = 1)
    output_df.to_csv(f'{out_dir}/Ent_space_{lfreq}_{hfreq}.csv', index=False, sep=',')

    # save space
    output_df_space = {'sub':sub_all, 'day': day_all,'condition':condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(LZC_space).reset_index(drop=True)), axis = 1)
    output_df.to_csv(f'{out_dir}/LZC_space_{lfreq}_{hfreq}.csv', index=False, sep=',')

    # save space
    output_df_space = {'sub':sub_all, 'day': day_all,'condition':condition_all}
    output_df = pd.concat((pd.DataFrame(output_df_space), pd.DataFrame(KDF_space).reset_index(drop=True)), axis = 1)
    output_df.to_csv(f'{out_dir}/KDF_space_{lfreq}_{hfreq}.csv', index=False, sep=',')


Analyzing Lyapunov Exponenf of data/input/10s/sub0-day1-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:48: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:54: RuntimeWarning: filter_length (8449) is longer than the signal (2560), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt = epochs_mne.filter(lfreq, hfreq, verbose = False)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:58: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_filt.get_data()


Done Trial 8
Done Trial 4
Done Trial 11
Done Trial 7
Done Trial 5
Done Trial 6
Done Trial 3
Done Trial 0
Done Trial 2
Done Trial 10
Done Trial 1
Done Trial 9
Done Trial 12
Done Trial 14
Done Trial 13
Done Trial 16
Done Trial 15
Done Trial 22
Done Trial 18
Done Trial 17
Done Trial 20
Done Trial 21
Done Trial 19
Done Trial 23
Done Trial 26
Done Trial 24
Done Trial 25
Done Trial 27
Done Trial 28
Done Trial 29
Analyzing Lyapunov Exponenf of data/input/10s/sub0-day1-mindfulness-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-mindfulness-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:45: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  epochs_mne.interpolate_bads(reset_bads=True)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:48: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:54: RuntimeWarning: filter_length (8449) is longer than the signal (2560), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filt = epochs_mne.filter(lfreq, hfreq, verbose = False)
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_50356/1925645566.py:58: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_filt.get_data()


Done Trial 3
Done Trial 6
Done Trial 1
Done Trial 10
Done Trial 4
Done Trial 7
Done Trial 9
Done Trial 8
Done Trial 2
Done Trial 5
Done Trial 0
Done Trial 11
Done Trial 19
Done Trial 16
Done Trial 12
Done Trial 15
Done Trial 14
Done Trial 18
Done Trial 21
Done Trial 22
Done Trial 23
Done Trial 20
Done Trial 17
Done Trial 13
Done Trial 27
Done Trial 25
Done Trial 29
Done Trial 26
Done Trial 24
Done Trial 28
